In [6]:
import pandas as pd

fuel_total_df = pd.read_csv('데이터/한국남동발전_연료소비실적(월평균).csv', encoding='utf-8-sig')
generation_df = pd.read_csv('데이터/한국남동발전_발전실적.csv', encoding='utf-8-sig')

fuel_total_df['월'] = fuel_total_df['일자'].astype(str).str[:6]
generation_df['월'] = generation_df['일자'].astype(str).str[:6]

# 발전소 + 월별 총 발전량 계산
total_generation = generation_df.groupby(['사업소', '월'])['발전량(MWh)'].sum().reset_index()
total_generation = total_generation.rename(columns={'발전량(MWh)': '총발전량'})

# 발전실적에 총발전량 병합
generation_df = generation_df.merge(total_generation, on=['사업소', '월'], how='left')

# 각 호기의 발전량 비율 계산
generation_df['발전비율'] = generation_df['발전량(MWh)'] / generation_df['총발전량']

fuel_total_df = fuel_total_df.drop(columns=['호기'])

merged = generation_df.merge(fuel_total_df, on=['사업소', '월'], how='left')

for fuel in ['유연탄', '무연탄', '계(석탄)', '유류', 'LNG', '고형연료', '우드펠릿']:
    merged[f'{fuel}_호기별'] = merged['발전비율'] * merged[fuel]

# 필요한 열만 출력
result = merged[['사업소', '호기', '월'] + [f'{fuel}_호기별' for fuel in ['유연탄', '무연탄', '계(석탄)', '유류', 'LNG', '고형연료', '우드펠릿']]]

print(result)
result.to_csv('호기별_연료소비_추정.csv', index=False, encoding='utf-8-sig')


    사업소  호기       월        유연탄_호기별  무연탄_호기별      계(석탄)_호기별      유류_호기별  \
0    분당   1  202412       0.000000      0.0       0.000000    0.000000   
1    분당   2  202412       0.000000      0.0       0.000000    0.000000   
2    분당   3  202412       0.000000      0.0       0.000000    0.000000   
3    분당   4  202412       0.000000      0.0       0.000000    0.000000   
4    분당   5  202412       0.000000      0.0       0.000000    0.000000   
..   ..  ..     ...            ...      ...            ...         ...   
523  영흥   2  202301       0.000000      0.0       0.000000    0.000000   
524  영흥   3  202301  209758.169604      0.0  209758.169604  196.195856   
525  영흥   4  202301       0.000000      0.0       0.000000    0.000000   
526  영흥   5  202301  228209.511776      0.0  228209.511776  213.454192   
527  영흥   6  202301  230236.318621      0.0  230236.318621  215.349952   

          LNG_호기별  고형연료_호기별  우드펠릿_호기별  
0    10923.353216       0.0       0.0  
1     5522.410270       0.0    

In [14]:
#데이터 소수 둘째자리 반올림
import pandas as pd

file_path = '데이터/호기별_연료소비_추정.csv'

df = pd.read_csv(file_path, encoding='utf-8-sig')

float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].round(2)

print(df.head())

df.to_csv('데이터/호기별_연료소비_추정.csv', index=False, encoding='utf-8-sig')
print("저장 완료: 호기별_연료소비_추정.csv")

  사업소  호기       월  유연탄_호기별  무연탄_호기별  계(석탄)_호기별  유류_호기별   LNG_호기별  고형연료_호기별  \
0  분당   1  202412      0.0      0.0        0.0     0.0  10923.35       0.0   
1  분당   2  202412      0.0      0.0        0.0     0.0   5522.41       0.0   
2  분당   3  202412      0.0      0.0        0.0     0.0  13126.97       0.0   
3  분당   4  202412      0.0      0.0        0.0     0.0    498.93       0.0   
4  분당   5  202412      0.0      0.0        0.0     0.0   7888.54       0.0   

   우드펠릿_호기별  
0       0.0  
1       0.0  
2       0.0  
3       0.0  
4       0.0  
저장 완료: 호기별_연료소비_추정.csv
